# ボールアニメーション

In [1]:
import time
import numpy as np
from threading import Timer
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display as IPython_display
from ipycanvas import Canvas, hold_canvas
import math
from math import pi

In [2]:
# タイマークラス
class RepeatedTimer(Timer):
    def __init__(self, interval, function, args=[], kwargs={}):
        Timer.__init__(self, interval, self.run, args, kwargs)
        self.thread = None
        self.function = function

    def run(self):
        self.thread = Timer(self.interval, self.run)
        self.thread.start()
        self.function(*self.args, **self.kwargs)

    def cancel(self):
        if self.thread is not None:
            self.thread.cancel()
            self.thread.join()
            del self.thread

In [44]:
# ゲームオブジェクトクラス
class GameObject2D:
    def __init__(self, init_x=0, init_y=0, width=0, height=0, color='black', vec_x=0, vec_y=0):
        self.init_x = init_x
        self.init_y = init_y
        self.pos_x = init_x
        self.pos_y = init_y
        self.width = width
        self.height = height
        self.color = color
        self.vec_x = vec_x
        self.vec_y = vec_y

    def update(self):
        pass

# ボールクラス
class Ball(GameObject2D):
    def __init__(self, init_x=0, init_y=0, width=0, color='black', vec_x=0, vec_y=0):
        GameObject2D.__init__(self, init_x, init_y, width, width, color, vec_x, vec_y) # ボールはwidthだけ使用するのでheightにもwidthを入れる
        self.half_w = self.width * 0.5

    ## ボールをcanvas内に収める処理
    def _crop_canvas(self):
        if self.pos_x + self.half_w > canvas.width:
            self.pos_x = canvas.width - self.half_w  # 壁にくっつける
            self.vec_x = -self.vec_x
        elif self.pos_x - self.half_w < 0:
            self.pos_x = self.half_w  # 壁にくっつける
            self.vec_x = -self.vec_x

        if self.pos_y + self.half_w > canvas.height:
            self.pos_y = canvas.height - self.half_w  # 壁にくっつける
            self.vec_y = -self.vec_y
        elif self.pos_y - self.half_w < 0:
            self.pos_y = self.half_w  # 壁にくっつける
            self.vec_y = -self.vec_y

    ## ボール同士の当たり判定
    def _hit_one_ball(self, ball):
        sub_x = ball.pos_x - self.pos_x 
        sub_y = ball.pos_y - self.pos_y
        sub_r = math.hypot(sub_x, sub_y)
        concat_w = self.half_w + ball.half_w
        
        if sub_r < concat_w:
            vec_r = math.hypot(self.vec_x, self.vec_y)
            if vec_r > 0 and sub_r > 0:
                print(f'old pos_x={self.pos_x}, pos_y={self.pos_y}, ball_x={ball.pos_x}, ball_y={ball.pos_y}, vec_x={self.vec_x}, vec_y={self.vec_y}, sub_x={sub_x}, sub_y={sub_y}')
                self.pos_x = ball.pos_x - (sub_x / sub_r) * concat_w  # ボールにくっつける
                self.pos_y = ball.pos_y - (sub_y / sub_r) * concat_w  # ボールにくっつける
                self.vec_x = -(sub_x / sub_r) * vec_r
                self.vec_y = -(sub_y / sub_r) * vec_r
                print(f'new pos_x={self.pos_x}, pos_y={self.pos_y}, ball_x={ball.pos_x}, ball_y={ball.pos_y}, vec_x={self.vec_x}, vec_y={self.vec_y}')
            
    ## ボール同士の当たり判定
    def _hit_balls(self):
        for one_ball in balls:
            if one_ball != self:
                self._hit_one_ball(one_ball)
    
    def update(self):
        ### 速度を加算して位置を移動
        self.pos_x += self.vec_x
        self.pos_y += self.vec_y

        self._crop_canvas() ## ボールをcanvas内に収める
        self._hit_balls() ## ボール同士の当たり判定

        canvas.fill_style = self.color
        canvas.fill_arc(self.pos_x, self.pos_y, self.width, 0, 2 * pi)


In [48]:
# Canvas作成
canvas = Canvas(width=400, height=200)
canvas.stroke_rect(0, 0, canvas.width, canvas.height)
canvas

Canvas(height=200, width=400)

In [49]:
# 簡単なアニメーション実装
balls = [Ball(0,50,6,'green',5,1), Ball(0,100,6,'blue',5,1), Ball(0,150,6,'red',5,1), Ball(0,200,6,'pink',5,1), Ball(50,50,6,'yellow',5,1)]

## 毎フレームごとの更新処理
def on_update(hoge):
    with hold_canvas(canvas):
        canvas.clear_rect(0, 0, canvas.width, canvas.height)
        canvas.stroke_rect(0, 0, canvas.width, canvas.height)
        for one_ball in balls:
            one_ball.update()

## タイマー生成＆繰り返し
timer_interval = 0.02
measurement_duration = 15.0

t = RepeatedTimer(timer_interval, on_update, ["hoge"])
t.start()
time.sleep(measurement_duration)
t.cancel()

old pos_x=110, pos_y=176.0, ball_x=110, ball_y=172, vec_x=5, vec_y=-1, sub_x=0, sub_y=-4.0
new pos_x=110.0, pos_y=178.0, ball_x=110, ball_y=172, vec_x=-0.0, vec_y=5.0990195135927845
old pos_x=350, pos_y=175.0, ball_x=350, ball_y=170, vec_x=5, vec_y=-1, sub_x=0, sub_y=-5.0
new pos_x=350.0, pos_y=176.0, ball_x=350, ball_y=170, vec_x=-0.0, vec_y=5.0990195135927845
old pos_x=375, pos_y=125, ball_x=377.0, ball_y=124, vec_x=5, vec_y=1, sub_x=2.0, sub_y=-1
new pos_x=371.6334368540005, pos_y=126.68328157299975, ball_x=377.0, ball_y=124, vec_x=-4.560701700396551, vec_y=2.2803508501982757
old pos_x=372.0, pos_y=125, ball_x=371.6334368540005, ball_y=126.68328157299975, vec_x=-5, vec_y=1, sub_x=-0.3665631459995211, sub_y=1.6832815729997463
new pos_x=372.9101176908984, pos_y=120.82068095205534, ball_x=371.6334368540005, ball_y=126.68328157299975, vec_x=1.0849700833287246, vec_y=-4.98225249443278
old pos_x=346.00640608355116, pos_y=162.1242325389706, ball_x=350.0, ball_y=161.0696049213763, vec_x=-1.